# Importing libraries

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
import pandas as pd
import time

# Extracting jobs links 

In [16]:

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import gspread
from google.oauth2.service_account import Credentials
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from googleapiclient.discovery import build
import re
from urllib.parse import urlparse, urlunparse

# Set the path to the chromedriver executable
PATH = r"C:\Program Files (x86)\chromedriver.exe"
s = Service(PATH)

# Initialize the Chrome WebDriver
driver = webdriver.Chrome(service=s)

def clean_link(link):
    if '://www.' in link:
        return link
    pos = link.find('://') + 3
    base_part = link[pos:]
    if '.' in base_part and not base_part.startswith('www.'):
        first_dot = base_part.find('.')
        cleaned_url = link[:pos] + base_part[first_dot + 1:]
        return cleaned_url
    return link

def extract_part_from_url(url):
    parsed_url = urlparse(url)
    cleaned_url = urlunparse((parsed_url.scheme, parsed_url.netloc, parsed_url.path, '', '', ''))
    return cleaned_url

# Open the webpage
urle = "https://www.linkedin.com/jobs/search?keywords=Remote&location=france&geoId=&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0"
driver.get(urle)
time.sleep(3)

from urllib.parse import urlparse, parse_qs

def extract_location(url):
    # Parse the URL
    parsed_url = urlparse(url)
    
    # Extract the query parameters
    query_params = parse_qs(parsed_url.query)
    
    # Get the location parameter (after 'l=')
    location = query_params.get('location', [None])[0]
    
    return location

country = extract_location(urle)


# Parameters
initial_scrolls = 1
scroll_repeat_after_button = 1
max_clicks = 0
clicks = 0
consecutive_no_loads = 0
max_no_load_attempts = 4

# Initial scrolling to load content before interacting with the "Load More" button
for _ in range(initial_scrolls):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    WebDriverWait(driver, 2).until(lambda d: d.execute_script("return document.readyState") == "complete")
    time.sleep(2)

while clicks < max_clicks:
    try:
        load_more = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CLASS_NAME, "infinite-scroller__show-more-button")))
        driver.execute_script("arguments[0].scrollIntoView(true);", load_more)
        time.sleep(2)
        load_more.click()

        new_content_height = driver.execute_script("return document.body.scrollHeight")
        WebDriverWait(driver, 2).until(lambda d: d.execute_script("return document.body.scrollHeight") > new_content_height)
        print(f"Clicked 'Load More' {clicks + 1} times.")
        clicks += 1
        consecutive_no_loads = 0
        
        for _ in range(scroll_repeat_after_button):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            WebDriverWait(driver, 2).until(lambda d: d.execute_script("return document.readyState") == "complete")
            time.sleep(2)

    except (NoSuchElementException, TimeoutException) as e:
        print(f"No more 'Load More' button found or exception occurred: {e}")

        while True:
            try:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                WebDriverWait(driver, 2).until(lambda d: d.execute_script("return document.readyState") == "complete")
                time.sleep(2)
                new_content_loaded = driver.execute_script("return document.body.scrollHeight")
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                WebDriverWait(driver, 2).until(lambda d: d.execute_script("return document.readyState") == "complete")
                time.sleep(2)
                new_content_height = driver.execute_script("return document.body.scrollHeight")

                if new_content_height > new_content_loaded:
                    print("New content loaded, scrolling down again.")
                    consecutive_no_loads = 0
                    continue

                driver.execute_script("window.scrollBy(0, -1000);")
                WebDriverWait(driver, 2).until(lambda d: d.execute_script("return document.readyState") == "complete")
                time.sleep(2)
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                WebDriverWait(driver, 2).until(lambda d: d.execute_script("return document.readyState") == "complete")
                time.sleep(2)
                new_content_height = driver.execute_script("return document.body.scrollHeight")

                if new_content_height > new_content_loaded:
                    print("New content loaded after scrolling up, continuing.")
                    consecutive_no_loads = 0
                    continue

                load_more = WebDriverWait(driver, 4).until(EC.element_to_be_clickable((By.CLASS_NAME, "infinite-scroller__show-more-button")))
                driver.execute_script("arguments[0].scrollIntoView(true);", load_more)
                load_more.click()

                new_content_height = driver.execute_script("return document.body.scrollHeight")
                WebDriverWait(driver, 2).until(lambda d: d.execute_script("return document.body.scrollHeight") > new_content_height)
                print(f"Clicked 'Load More' {clicks + 1} times after retrying.")
                clicks += 1
                consecutive_no_loads = 0
                
                for _ in range(scroll_repeat_after_button):
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    WebDriverWait(driver, 2).until(lambda d: d.execute_script("return document.readyState") == "complete")
                    time.sleep(2)
                break

            except (NoSuchElementException, TimeoutException) as e:
                print(f"Retrying to find the 'Load More' button. Exception: {e}")
                consecutive_no_loads += 1
                if consecutive_no_loads >= max_no_load_attempts:
                    print("Exceeded maximum number of attempts without loading new content. Exiting loop.")
                    break

    if consecutive_no_loads >= max_no_load_attempts:
        print("Exceeded maximum number of attempts without loading new content. Exiting main loop.")
        break

scope = ['https://www.googleapis.com/auth/spreadsheets', "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]
creds_file = 'credentials.json'
creds = Credentials.from_service_account_file(creds_file, scopes=scope)
client = gspread.authorize(creds)
drive_service = build('drive', 'v3', credentials=creds)

spreadsheet_name = 'Linkedin'

def open_or_create_spreadsheet(name):
    try:
        return client.open(name)
    except gspread.exceptions.SpreadsheetNotFound:
        spreadsheet = client.create(name)
        file_id = spreadsheet.id
        move_file_to_folder(file_id, folder_id)
        return client.open_by_key(file_id)

def move_file_to_folder(file_id, folder_id):
    drive_service.files().update(
        fileId=file_id,
        addParents=folder_id,
        removeParents='',
        fields='id, parents'
    ).execute()

spreadsheet = open_or_create_spreadsheet(spreadsheet_name)
sheet2 = spreadsheet.worksheet("Feuille 2")
sheet3 = spreadsheet.worksheet("Feuille 3")

liste = []
comparing_list = []

try:
    container = driver.find_element(By.CLASS_NAME, 'jobs-search__results-list')
    links = container.find_elements(By.TAG_NAME, "li")
    for element in links:
        link = clean_link(element.find_element(By.TAG_NAME, "a").get_attribute("href"))
        cleanlink = extract_part_from_url(link)
        if cleanlink not in comparing_list:
            liste.append(link)
            comparing_list.append(cleanlink)

    df = pd.DataFrame({'Links': liste})
    set_with_dataframe(sheet3, df)  # Save links to Feuille 3

    comparing_list = [extract_part_from_url(link) for link in df['Links']]
    df_cleaned = pd.DataFrame({'Links': comparing_list})

    existing_links_df = get_as_dataframe(sheet2)
    combined_df = pd.concat([existing_links_df, df_cleaned], ignore_index=True).drop_duplicates().reset_index(drop=True)

    sheet2.clear()
    set_with_dataframe(sheet2, combined_df)  # Save combined DataFrame to Feuille 2

    print(f"Data successfully saved to 'linkedin' in Feuille 2 and Feuille 3.")
except Exception as e:
    print(f'Error in getting links: {e}')

driver.quit()


Data successfully saved to 'linkedin' in Feuille 2 and Feuille 3.


# Ectracting jobs informations

In [17]:

import subprocess
import time
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from gspread_dataframe import get_as_dataframe, set_with_dataframe
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Authentification Google Sheets
scope = ["https://spreadsheets.google.com/feeds", 'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('credentials.json', scope)
client = gspread.authorize(creds)

# Définir le chemin vers le ChromeDriver
PATH = r"C:\Program Files (x86)\chromedriver.exe"
s = Service(PATH)
driver = webdriver.Chrome(service=s)

# Charger les données depuis la feuille 3 du fichier 'linkedin'
spreadsheet = client.open('Linkedin')  # Replace with your Google Sheet name
sheet = spreadsheet.get_worksheet(2)  # Sheet index is 0-based, so 2 means Sheet 3

# Read existing links from the sheet into a DataFrame
existing_df = get_as_dataframe(sheet)

# Extract the 'Links' column
links = existing_df['Links']

# Initialiser les listes pour stocker les données extraites
job_title = []
company_name = []
company_link = []
description = []
Niveau_hierarchique = []
Type_emploi = []
Fonction = []
Secteurs = []

# Dictionnaire pour stocker les données extraites par lien unique
extracted_data = {}

# Maximum number of retries for each link
MAX_RETRIES = 4

# Function to clean the link
def clean_link(link):
    if '://www.' in link:
        return link
    pos = link.find('://') + 3
    base_part = link[pos:]
    if '.' in base_part and not base_part.startswith('www.'):
        first_dot = base_part.find('.')
        cleaned_url = link[:pos] + base_part[first_dot + 1:]
        return cleaned_url
    return link

# Traitement des liens
for i, link in enumerate(links):
    # if i > 5:
    #     break
    retries = 0
    success = False

    while retries < MAX_RETRIES and not success:
        
        if link not in extracted_data:
            try:
                driver.get(link)
                time.sleep(2)
                print(f"Processing link {i}: {link}")

                WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.TAG_NAME, "body")))

                try:
                    container = WebDriverWait(driver, 2).until(
                        EC.presence_of_element_located((By.CLASS_NAME, "core-rail"))
                    )

                    TOP_card = container.find_element(By.CLASS_NAME, "top-card-layout")
                    job_title_value = TOP_card.find_element(By.CLASS_NAME, "top-card-layout__title").text
                    company_name_value = TOP_card.find_element(By.CLASS_NAME, "topcard__org-name-link").text
                    company_link_value = clean_link(TOP_card.find_element(By.CLASS_NAME, "topcard__org-name-link").get_attribute("href"))

                    details_container = container.find_element(By.CLASS_NAME, "core-section-container__content")
                    description_value = details_container.find_element(By.CLASS_NAME, "show-more-less-html__markup").text

                    job_criterias_container = details_container.find_element(By.CLASS_NAME, "description__job-criteria-list")
                    job_criteria_list = job_criterias_container.find_elements(By.CLASS_NAME, "description__job-criteria-item")

                    criteria_values = {
                        'Niveau_hierarchique': 'null',
                        'Type_emploi': 'null',
                        'Fonction': 'null',
                        'Secteurs': 'null'
                    }

                    for k, criteria in enumerate(job_criteria_list):
                        if k == 0:
                            criteria_values['Niveau_hierarchique'] = criteria.find_element(By.CLASS_NAME, 'description__job-criteria-text').text
                        elif k == 1:
                            criteria_values['Type_emploi'] = criteria.find_element(By.CLASS_NAME, 'description__job-criteria-text').text
                        elif k == 2:
                            criteria_values['Fonction'] = criteria.find_element(By.CLASS_NAME, 'description__job-criteria-text').text
                        elif k == 3:
                            criteria_values['Secteurs'] = criteria.find_element(By.CLASS_NAME, 'description__job-criteria-text').text
                    
                    extracted_data[link] = {
                        "job_title": job_title_value,
                        "company_name": company_name_value,
                        "company_link": company_link_value,
                        "description": description_value,
                        "Niveau_hierarchique": criteria_values['Niveau_hierarchique'],
                        "Type_emploi": criteria_values['Type_emploi'],
                        "Fonction": criteria_values['Fonction'],
                        "Secteurs": criteria_values['Secteurs']
                    }
                    success = True

                except Exception as e:
                    print(f"Erreur lors de l'extraction des données pour le lien {link}: {e}")
                    retries += 1

            except Exception as e:
                print(f"Erreur lors du traitement du lien {link}: {e}")
                retries += 1
               
            if retries == MAX_RETRIES:
                extracted_data[link] = {
                    "job_title": "null",
                    "company_name": "null",
                    "company_link": "null",
                    "description": "null",
                    "Niveau_hierarchique": "null",
                    "Type_emploi": "null",
                    "Fonction": "null",
                    "Secteurs": "null"
                }

    data = extracted_data[link]
    job_title.append(data["job_title"])
    company_name.append(data["company_name"])
    company_link.append(data["company_link"])
    description.append(data["description"])
    Niveau_hierarchique.append(data["Niveau_hierarchique"])
    Type_emploi.append(data["Type_emploi"])
    Fonction.append(data["Fonction"])
    Secteurs.append(data["Secteurs"])

# Assurer que toutes les listes ont la même longueur
max_length = max(len(job_title), len(company_name), len(company_link), len(description), len(Niveau_hierarchique), len(Type_emploi), len(Fonction), len(Secteurs))

for field in [job_title, company_name, company_link, description, Niveau_hierarchique, Type_emploi, Fonction, Secteurs]:
    while len(field) < max_length:
        field.append("null")

# Convertir les listes en DataFrame
df = pd.DataFrame({
    'Nom entreprise': company_name,
    'Lien entreprise': company_link,
    "Secteur d'emploi": Secteurs,
    "Offre emploi": job_title,
    'Description emploi': description,
    'Niveau hierarchique': Niveau_hierarchique,
    'Type emploi': Type_emploi,
    'Fonction': Fonction,
})

# Écrire les données extraites dans la feuille 3 du fichier 'linkedin'
set_with_dataframe(sheet, df)

print("Extraction et mise à jour des données terminées.")
driver.quit()


Processing link 0: https://linkedin.com/jobs/view/corsican-transcribers-remote-at-sigma-ai-3989185250?position=1&pageNum=0&refId=k%2Bdj0Fuhb7INE7yszoej4g%3D%3D&trackingId=%2FqokIXKHdWUBuLZMl3WmxA%3D%3D&trk=public_jobs_jserp-result_search-card
Processing link 1: https://linkedin.com/jobs/view/reels-creator-remote-at-metaschool-4002317646?position=2&pageNum=0&refId=k%2Bdj0Fuhb7INE7yszoej4g%3D%3D&trackingId=myhkuHX%2FHmUXqiw%2B6iLRsA%3D%3D&trk=public_jobs_jserp-result_search-card
Processing link 2: https://linkedin.com/jobs/view/events-coordinator-remote-at-thermo-fisher-scientific-3996892084?position=3&pageNum=0&refId=k%2Bdj0Fuhb7INE7yszoej4g%3D%3D&trackingId=PmO%2FDSG6%2F9IdVHOchu8tGw%3D%3D&trk=public_jobs_jserp-result_search-card
Processing link 3: https://linkedin.com/jobs/view/powerpoint-creative-100%25-remote-at-von-peach-3997760885?position=4&pageNum=0&refId=k%2Bdj0Fuhb7INE7yszoej4g%3D%3D&trackingId=2j%2BKMmFt7YGWbbxSLs1OFw%3D%3D&trk=public_jobs_jserp-result_search-card
Processing 

# Manipulation des données 

Anuller les NULL et trier alphabétiquement

In [5]:
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import get_as_dataframe, set_with_dataframe

# Authentification Google Sheets
scope = ["https://spreadsheets.google.com/feeds", 'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('credentials.json', scope)
client = gspread.authorize(creds)

# Charger les données depuis la feuille 3 du fichier 'linkedin'
spreadsheet = client.open('Linkedin')  # Remplacez par le nom de votre Google Sheet
sheet = spreadsheet.get_worksheet(2)  # La feuille 3 (l'index est basé à 0)

# Lire les données de la feuille 3 dans un DataFrame
df = get_as_dataframe(sheet)

# Supprimer les lignes où toutes les valeurs sont 'null'
df = df[~df[["Offre emploi", 'Nom entreprise', 'Lien entreprise', "Description emploi", 'Niveau hierarchique', 'Type emploi', 'Fonction', "Secteur d'emploi"]].eq('null').all(axis=1)]

# Trier le DataFrame par la colonne "company name" alphabétiquement
df = df.sort_values(by='Nom entreprise')

# Écrire les résultats mis à jour dans la feuille 3 de Google Sheets
set_with_dataframe(sheet, df)

print("Mise à jour de la feuille Google Sheet terminée.")


Mise à jour de la feuille Google Sheet terminée.


# Extracting companies informations 

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
import pandas as pd
import time
import re
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import get_as_dataframe, set_with_dataframe

# Authentification Google Sheets
scope = ["https://spreadsheets.google.com/feeds", 'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('credentials.json', scope)
client = gspread.authorize(creds)

# Charger les données depuis la feuille 3 de Google Sheets
spreadsheet = client.open('Linkedin')
sheet_3 = spreadsheet.get_worksheet(2)  # La feuille 3 (l'index est basé à 0)

# Lire les données de la feuille 3 dans un DataFrame
original_data = get_as_dataframe(sheet_3).fillna('')
links = original_data['Lien entreprise']

# Initialiser les listes pour stocker les données extraites
description = []
site_web = []
secteur = []
taille_entreprise = []
siege_social = []
type_entreprise = []
domaines = []
annee_fondation = []

# Dictionnaire pour stocker les données extraites par lien unique
extracted_data = {}

def normalize_text(text):
    # Supprimer les caractères non alphanumériques et normaliser les espaces
    return re.sub(r'\W+', '', text).strip().lower()

# Chemin vers ChromeDriver
PATH = r"C:\Program Files (x86)\chromedriver.exe"
s = Service(PATH)

# Nombre maximum de tentatives en cas d'erreur
max_retries = 3

for i, link in enumerate(links):
    attempt = 0
    success = False

    while attempt < max_retries and not success:
        try:
            if link not in extracted_data:
                # Initialiser le WebDriver pour chaque tentative
                driver = webdriver.Chrome(service=s)
                driver.get(link)
                print("company_____",i)
                WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.TAG_NAME, "body")))  # Attendre que la page soit complètement chargée

                # Gérer le bouton de fermeture de la fenêtre modale si présent
                try:
                    button = WebDriverWait(driver, 8).until(
                        EC.element_to_be_clickable((By.CLASS_NAME, "modal__dismiss"))
                    )
                    button.click()
                except Exception as e:
                    print(f"Aucun bouton de fermeture de modal trouvé: {e}")

                # Extraire la description
                try:
                    description_element = WebDriverWait(driver, 8).until(
                        EC.visibility_of_element_located((By.CSS_SELECTOR, '[data-test-id="about-us__description"]'))
                    )
                    description_value = description_element.text
                except Exception as e:
                    print(f"Erreur lors de l'extraction de la description: {e}")
                    description_value = "null"

                # Initialiser les valeurs par défaut pour les champs
                site_web_value = "null"
                secteur_value = "null"
                taille_entreprise_value = "null"
                siege_social_value = "null"
                type_entreprise_value = "null"
                domaines_value = "null"
                annee_fondation_value = "null"

                # Extraire plus de détails
                try:
                    more_details_container = WebDriverWait(driver, 10).until(
                        EC.visibility_of_element_located((By.CLASS_NAME, "mt-6"))
                    )

                    dt_container = more_details_container.find_elements(By.TAG_NAME, 'dt')
                    dd_container = more_details_container.find_elements(By.TAG_NAME, 'dd')

                    # Dictionnaires pour stocker les champs et leurs valeurs extraites
                    fields = {
                        normalize_text("Site web"): site_web_value,
                        normalize_text("Secteur"): secteur_value,
                        normalize_text("Taille de l'entreprise"): taille_entreprise_value,
                        normalize_text("Siège social"): siege_social_value,
                        normalize_text("Type"): type_entreprise_value,
                        normalize_text("Domaines"): domaines_value,
                        normalize_text("Fondée en"): annee_fondation_value
                    }

                    # Extraire les données basées sur les paires dt/dd
                    for dt, dd in zip(dt_container, dd_container):
                        key = normalize_text(dt.text)
                        value = dd.text.strip()
                        if key in fields:
                            fields[key] = value
                        else:
                            print(f"Champ non reconnu: {dt.text.strip()}")

                    # Extraire les valeurs
                    site_web_value = fields[normalize_text("Site web")]
                    secteur_value = fields[normalize_text("Secteur")]
                    taille_entreprise_value = fields[normalize_text("Taille de l'entreprise")]
                    siege_social_value = fields[normalize_text("Siège social")]
                    type_entreprise_value = fields[normalize_text("Type")]
                    domaines_value = fields[normalize_text("Domaines")]
                    annee_fondation_value = fields[normalize_text("Fondée en")]

                except Exception as e:
                    print(f"Erreur lors de l'extraction des détails: {e}")

                # Stocker les valeurs extraites dans le dictionnaire
                extracted_data[link] = {
                    "description": description_value,
                    "site_web": site_web_value,
                    "secteur": secteur_value,
                    "taille_entreprise": taille_entreprise_value,
                    "siege_social": siege_social_value,
                    "type_entreprise": type_entreprise_value,
                    "domaines": domaines_value,
                    "annee_fondation": annee_fondation_value
                }

                # Fermer la fenêtre actuelle
                driver.quit()
                time.sleep(1)

            success = True

        except Exception as e:
            print(f"Erreur lors de l'extraction des données pour le lien {link}: {e}")
            attempt += 1
            # Fermer le WebDriver en cas d'échec pour réinitialiser l'état
            driver.quit()
            time.sleep(1)

            if attempt >= max_retries:
                # Stocker des valeurs "null" en cas d'échec après plusieurs tentatives
                extracted_data[link] = {
                    "description": "null",
                    "site_web": "null",
                    "secteur": "null",
                    "taille_entreprise": "null",
                    "siege_social": "null",
                    "type_entreprise": "null",
                    "domaines": "null",
                    "annee_fondation": "null"
                }
                break

    # Utiliser les données extraites pour ce lien
    data = extracted_data[link]
    description.append(data["description"])
    site_web.append(data["site_web"])
    secteur.append(data["secteur"])
    taille_entreprise.append(data["taille_entreprise"])
    siege_social.append(data["siege_social"])
    type_entreprise.append(data["type_entreprise"])
    domaines.append(data["domaines"])
    annee_fondation.append(data["annee_fondation"])

# Vérifier les longueurs des listes
lengths = [len(description), len(site_web), len(secteur), len(taille_entreprise), len(siege_social), len(type_entreprise), len(domaines), len(annee_fondation)]
max_length = max(lengths)

# Ajouter des valeurs "null" pour égaliser les longueurs des listes
for field in [description, site_web, secteur, taille_entreprise, siege_social, type_entreprise, domaines, annee_fondation]:
    while len(field) < max_length:
        field.append("null")

# Convertir les listes en DataFrame
scraped_data = pd.DataFrame({
    "Description entreprise": description,
    'Site web': site_web,
    'Secteur': secteur,
    "Taille entreprise": taille_entreprise,
    'Siège social': siege_social,
    'Type': type_entreprise,
    'Domaines': domaines,
    "Année de fondation": annee_fondation
})

# Concaténer les données originales et les nouvelles données
scraped_data['Pays'] = country

combined_data = pd.concat([original_data.reset_index(drop=True), scraped_data.reset_index(drop=True)], axis=1)

sheet_1 = spreadsheet.get_worksheet(0)  # La feuille 1 (l'index est basé à 0)

# Écrire les données combinées dans la feuille 1
set_with_dataframe(sheet_1, combined_data)


company_____ 0
company_____ 1
company_____ 2
company_____ 3
company_____ 4
company_____ 5
company_____ 7
company_____ 9
company_____ 10
company_____ 11
company_____ 13
company_____ 15
company_____ 16
company_____ 17
company_____ 18
company_____ 19
company_____ 23
company_____ 24
company_____ 25
company_____ 28
company_____ 31
company_____ 32
company_____ 37
company_____ 39
Erreur lors de l'extraction de la description: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF61D07EEA2+31554]
	(No symbol) [0x00007FF61CFF7ED9]
	(No symbol) [0x00007FF61CEB872A]
	(No symbol) [0x00007FF61CF08434]
	(No symbol) [0x00007FF61CF0853C]
	(No symbol) [0x00007FF61CF4F6A7]
	(No symbol) [0x00007FF61CF2D06F]
	(No symbol) [0x00007FF61CF4C977]
	(No symbol) [0x00007FF61CF2CDD3]
	(No symbol) [0x00007FF61CEFA33B]
	(No symbol) [0x00007FF61CEFAED1]
	GetHandleVerifier [0x00007FF61D388B1D+3217341]
	GetHandleVerifier [0x00007FF61D3D5AE3+3532675]
	GetHandleVerifier [0x00007FF61D3CB0E0+3489152]
	GetHandleVerifier [0x000

In [19]:
import pandas as pd


# Créer une fonction pour mapper les secteurs aux catégories
def map_to_category(sector):
    if sector in [
    'Sécurité informatique et des réseaux',
    'Développement de logiciels',
    'Technologies et services de l’information',
    'Services et conseil en informatique',
    'Télécommunications',
    'Technologie, information et Internet',
    'Produits de réseaux informatiques',
    'Fabrication de produits informatiques et électroniques',
    'Services de marketing',
    'Jeux vidéo',
    'Services de blockchain'
]:
        return 'IT'
    elif sector in [
        'Services bancaires', 'Activités de placement pour compte propre',
        'Services financiers', 'Assurances', 'Comptabilité'
    ]:
        return 'Finance'
    elif sector in [
    'Services de bien-être et installations sportives',
    'Hôpitaux et services de santé',
    'Soins de santé mentale',
    'Fabrication d’équipements médicaux',
    'Recherche en biotechnologie',
    'Cosmétiques'
]:
        return 'Santé'
    elif sector in [
    'Programmes d’administration de l’éducation',
    'Enseignement primaire et secondaire',
    'Enseignement supérieur',
    'Enseignement',
    'Fournisseurs d’apprentissage en ligne'
]:
        return 'Éducation'
    elif sector in [
        'Publicité', 'Marketing', 'Design graphique', 'Services de design','Services de publicité'
    ]:
        return 'Marketing & Design'
    elif sector in [
        'Hôtellerie', 'Organisation de voyages', 'Services de restauration'
    ]:
        return 'Hôtellerie & Restauration'
    elif sector in [
        'Immobilier non résidentiel locatif'
    ]:
        return 'Immobilier'
    elif sector in[
    'Environnement et énergies renouvelables',
    'Production d’énergies renouvelables',
    'Services pour les énergies renouvelables',
    'Données et analyses climatiques'
    ]:
        return 'Énergie et Environnement'

    elif sector in [
    'Vente à distance',
    'Commerce de gros de matériaux de construction',
    'Commerce de détail d’habillement et d’accessoires de mode',
    'Commerce de détail',
    'Articles de luxe et bijouterie',
    'Transport, logistique, chaîne logistique et stockage',
    'Transport maritime',
    'Transport aérien'
]:
        return 'Commerce & Logistique'
    elif sector in [
    'Fabrication d’appareils électroménagers, électriques et électroniques',
    'Fabrication de semi-conducteurs pour énergies renouvelables',
    'Fabrication de semi-conducteurs',
    'Fabrication de machines',
    'Fabrication de produits alimentaires et boissons',
    'Fabrication de parfums et de produits pour la toilette',
    'Fabrication de produits informatiques et électroniques',
    'Fabrication de composants pour l’industrie aéronautique et aérospatiale',
    'Fabrication de verre, de produits céramiques et de ciment',
    'Industrie manufacturière',
    'Automatismes industriels'
]:
        return 'Industrie'
    elif sector in [
        'Jeux vidéo', 'Médias audio et vidéo en ligne', 'Édition en ligne','Production audiovisuelle'
    ]:
        return 'Médias & Divertissement'
    elif sector in [
    'Recrutement et placement de personnel',
    'Services et conseil aux entreprises',
    'Think tanks',
    'Services financiers',
    'Services professionnels',
    'Services de ressources humaines',
    'Services de recrutement de cadres',
    'Services de systèmes de sécurité',
    'Services de conseil en environnement',
    'Services de recherche',
    'Services d’information',
    'Services de publicité',
    'Développement professionnel et coaching'
]:
        return 'Services aux entreprises'
    else:
        return 'Autres'

# Appliquer la fonction à chaque ligne du DataFrame
combined_data['Categorie'] = combined_data['Secteur'].apply(map_to_category)

desired_order = ['Offre emploi', 'Description emploi', 'Type emploi', 'Niveau hierarchique','Fonction', "Secteur d'emploi",'Nom entreprise',"Lien entreprise", "Description entreprise","Site web","Secteur","Taille entreprise",'Siège social','Type','Domaines','Année de fondation','Categorie','Pays']

# Reordering columns
combined_data = combined_data[desired_order]

In [20]:
sheets_service = build('sheets', 'v4', credentials=creds)

# Charger les données existantes de la feuille 1
sheet_1 = spreadsheet.get_worksheet(0)  # La feuille 1 (l'index est basé à 0)

# Écrire les données combinées dans la feuille 1
set_with_dataframe(sheet_1, combined_data)

print("Mise à jour de la feuille Google Sheets terminée.")

# Get the third sheet (index 2, as it is 0-based)
sheet = spreadsheet.get_worksheet(2)

# Clear the contents of the third sheet
sheet.clear()

print("Le contenu de la feuille 3 a été supprimé avec succès.")

Mise à jour de la feuille Google Sheets terminée.
Le contenu de la feuille 3 a été supprimé avec succès.


In [24]:
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import get_as_dataframe, set_with_dataframe
import re

# Define scope and credentials
scope = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('credentials.json', scope)
client = gspread.authorize(creds)

# Open the Google Sheet by name
sheet_name = 'Linkedin'
spreadsheet = client.open(sheet_name)

# Access "Feuille 1" for output
try:
    sheet1 = spreadsheet.worksheet('Feuille 1')
except gspread.exceptions.WorksheetNotFound:
    sheet1 = spreadsheet.add_worksheet(title='Feuille 1', rows=1000, cols=20)

# Read data from "Feuille 3" into a DataFrame
data = get_as_dataframe(sheet1)

# Define a function to process the "Taille entreprise" column
def process_size(size_str):
    if pd.isna(size_str) or size_str.strip() == "" or size_str.strip().lower() == "null":
        return 0

    # Remove all spaces within the size range
    size_str = re.sub(r'\s+', '', size_str)
    
    # Search for intervals
    match = re.search(r'(\d+)-(\d+)', size_str)
    if match:
        return int(match.group(2))
    
    # Search for single numbers
    match = re.search(r'(\d+)', size_str)
    if match:
        return int(match.group(1))
    
    return 0

# Add the new column "taille num" based on the "Taille entreprise" column
data['taille num'] = data['Taille entreprise'].apply(process_size)

# Trier les lignes par la colonne "taille num"
data = data.sort_values(by='taille num', ascending=True)

# Write the updated DataFrame to "Feuille 1"
set_with_dataframe(sheet1, data)
